In [1]:
import pandas as pd
import numpy as np

from train_test_split import split_data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import naivebayes

X_train, X_test, y_train, y_test = split_data()
scaler_minmax = MinMaxScaler()
scaler_standard = StandardScaler()
X_train_minmax = scaler_minmax.fit_transform(X_train)
X_test_minmax = scaler_minmax.fit_transform(X_test)
X_train_standard = scaler_standard.fit_transform(X_train)
X_test_standard = scaler_standard.fit_transform(X_test)

print("Running model with outliers present: ")

# Run gaussian naive bayes
y_pred = naivebayes.run_naive_bayes(X_train, X_test, y_train)
y_pred_minmax = naivebayes.run_naive_bayes(X_train_minmax, X_test_minmax, y_train)
y_pred_standard = naivebayes.run_naive_bayes(X_train_standard, X_test_standard, y_train)
acc = naivebayes.accuracy_scorer(y_test, y_pred)
acc_minmax = naivebayes.accuracy_scorer(y_test, y_pred_minmax)
acc_standard = naivebayes.accuracy_scorer(y_test, y_pred_standard)
print(f"Accuracy Gaussian Naive Bayes: {acc}")
print(f"Accuracy Gaussian Naive Bayes min max normalized: {acc_minmax}")
print(f"Accuracy Gaussian Naive Bayes standardized: {acc_standard}\n")

# Remove outliers
train_set = np.concatenate([X_train, y_train[:,None]], axis=1) # Add y_train array as last column of X_train 2d array
df = pd.DataFrame(train_set)                                   # Create pandas dataframe of features and labels
length = len(train_set[0]) - 1
for i in range(length):                                        # For each feature in the set, remove each row that has 
    df[i] = df[i].astype(int)                                  # at least one feature that is below the 1-percentile or
    q_u = df[i].quantile(0.99)                                 # it is above the 99th percentile
    q_l = df[i].quantile(0.01)
    df = df[df[i] <= q_u]
    df = df[df[i] >= q_l]

X_train = df.iloc[:, :-1].to_numpy()
y_train = df.iloc[:, -1].to_numpy()
X_train_minmax = scaler_minmax.fit_transform(X_train)
X_test_minmax = scaler_minmax.fit_transform(X_test)
X_train_standard = scaler_standard.fit_transform(X_train)
X_test_standard = scaler_standard.fit_transform(X_test)

print("Running model without outliers present: ")

# Run gaussian naive bayes
y_pred = naivebayes.run_naive_bayes(X_train, X_test, y_train)
y_pred_minmax = naivebayes.run_naive_bayes(X_train_minmax, X_test_minmax, y_train)
y_pred_standard = naivebayes.run_naive_bayes(X_train_standard, X_test_standard, y_train)
acc = naivebayes.accuracy_scorer(y_test, y_pred)
acc_minmax = naivebayes.accuracy_scorer(y_test, y_pred_minmax)
acc_standard = naivebayes.accuracy_scorer(y_test, y_pred_standard)
print(f"Accuracy Gaussian Naive Bayes: {acc}")
print(f"Accuracy Gaussian Naive Bayes min max normalized: {acc_minmax}")
print(f"Accuracy Gaussian Naive Bayes standardized: {acc_standard}")

Running models with outliers present: 
Accuracy Gaussian Naive Bayes: 0.6280162958320276
Accuracy Gaussian Naive Bayes min max normalized: 0.6320902538389219
Accuracy Gaussian Naive Bayes standardized: 0.6085866499529928

Running models without outliers present: 
Accuracy Gaussian Naive Bayes: 0.6505797555625196
Accuracy Gaussian Naive Bayes min max normalized: 0.642118458163585
Accuracy Gaussian Naive Bayes standardized: 0.61172046380445
